In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj3.ipynb")

# Project 3: Data Transformation

## Due Date: Friday 10/28, 11:59 PM

## Assignment Details

In this project, we'll be working with one month of data from sensors in buildings at UC Berkeley. This is a very typical real-world dataset---i.e. it's kind of a mess. The full dataset contains a giant `data` table of sensor readings that is many billions of readings over the course of a decade; we will look at a single month of that data. It also contains a variety of other tables that contextualize the readings.

The schema for the database is shown below. Sometimes people think that if the data is a nice schema, then it's ready to go! We'll see about that.

<img src="data/schema.png">

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests. Free-response questions will be manually graded.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
1a	| 1
1b  | 1
1c	| 1
1d	| 2
2a	| 3
2b	| 1
3a	| 0
3b	| 1
3c	| 1
3d  | 0
3e  | 2
4a	| 2
4b	| 2
4c	| 3
**Total** | 20

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/template1

## Loading Up the Database
To load the database, run the following cell.

**IMPORTANT NOTE:** In contrary to previous projects, the database does not reload automatically upon running the following loading cell. If you would like to reload the database (e.g. if your database is modified in undesirable ways), **first restart the kernel**, and then run `!psql -h localhost -c 'DROP DATABASE IF EXISTS ucb_buildings'` in another cell **before** running the cell below.

In [4]:
#!psql -h localhost -c 'DROP DATABASE IF EXISTS ucb_buildings'

In [5]:
import subprocess
import os
import warnings

call = subprocess.run(["psql", "-h", "localhost", \
                       "-tAc", "SELECT 1 FROM pg_database WHERE datname='ucb_buildings'", "template1"], \
                      stdout=subprocess.PIPE, text=True)

if call.stdout != "1\n":
    print("Loading database...")
    os.system("gunzip -c data/proj3.sql.gz | psql -h localhost -d template1 -f -")
else:
    print("Database was previously loaded.")

Database was previously loaded.


Now, run the following cell to connect to the `ucb_buildings` database. There should be no errors after running the following cell.

In [6]:
%sql postgresql://jovyan@127.0.0.1:5432/ucb_buildings

Run the following cell for grading purposes.

In [7]:
!mkdir -p results

<!-- BEGIN QUESTION -->

## Question 1: Unboxing the Data

### Question 1a

Note that the `data` table, in the full database, is billions of rows. What do you notice about the design of the database schema that helps support the large amount of data?

<!--
BEGIN QUESTION
name: q1a
manual: true
points: 1
-->

Despite the data table being billions of rows, having multiple tables that hold similar data significantly decreases the time it takes to query the data if the user knows exactly what they are looking for since normalization is being used to eliminate redundancy. 

<!-- END QUESTION -->



### Question 1b

The diagram claims that `buildings_site_mapping` has a many-to-many relationship with `real_estate_metadata`. Let's validate that. 

Below is an example of `json_agg` being used with a table; you will need to do this in the next two parts.

In [8]:
%%sql
SELECT b.site, json_agg(b) from buildings_site_mapping b GROUP BY b.site LIMIT 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
5 rows affected.


site,json_agg
Sutardja Dai Hall,"[{'site': 'Sutardja Dai Hall', 'building': 'SUTARDJA DAI'}]"
Le Conte Hall,"[{'site': 'Le Conte Hall', 'building': 'LE CONTE'}]"
Hildebrand Hall,"[{'site': 'Hildebrand Hall', 'building': 'HILDEBRA ND'}]"
Stephens Hall,"[{'site': 'Stephens Hall', 'building': 'STEPHENS'}]"
2000 Carleton Street,"[{'site': '2000 Carleton Street', 'building': 'CARLETO2 000'}]"


Find the **distinct** values of `buildings_site_mapping.building` that match multiple tuples in `real_estate_metadata.building_name`, and for each such value of `buildings_site_mapping.building` return the matches as JSON via `json_agg(real_estate_metadata)`. Your output should contain the building and the `json_agg` in that order. Order your final result by building.

**Hint:** You should use a CTE to find the distinct buildings of `buildings_site_mapping` before applying necessary table joins. The DISTINCT keyword does not achieve the desired effect.

In [9]:
# %%sql
# with cte as 
# (select DISTINCT building
#     from buildings_site_mapping)

# select cte.building, json_agg(r) from real_estate_metadata as r
# INNER JOIN cte on r.building_name = cte.building
# GROUP BY cte.building
# HAVING count(cte.building) > 1
# order by cte.building

interpret this as real_estate_md having 5 different places where the building names are the same. If building_name were a unique indentifier then this would have 0 rows

In [10]:
%%sql result_1b <<
with cte as 
(select DISTINCT building
    from buildings_site_mapping)

select cte.building, json_agg(r) from real_estate_metadata as r
INNER JOIN cte on r.building_name = cte.building
GROUP BY cte.building
HAVING count(cte.building) > 1
order by cte.building

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
5 rows affected.
Returning data to local variable result_1b


In [11]:
# Do not delete/edit this cell
result_1b.DataFrame().to_csv('results/result_1b.csv', index=False)

In [12]:
result_1b.DataFrame()

,building,json_agg
0,ALUMNI HOUSE,"[{'location': 'BERKELEY', 'building': '1215', ..."
1,CAMPBELL,"[{'location': 'BERKELEY', 'building': '1027', ..."
2,HERTZ,"[{'location': 'BERKELEY', 'building': '1423', ..."
3,SOUTH,"[{'location': 'BERKELEY', 'building': '1484', ..."
4,SPROUL,"[{'location': 'BERKELEY', 'building': '1210', ..."


In [13]:
grader.check("q1b")

q1b results: All test cases passed!

### Question 1c

Now find examples of many matches in the opposite direction. For each distinct `real_estate_metadata.building_name` value, find the ones that have multiple matches in `buildings_site_mapping.building`, and for each return a `json_agg` of the multiple values for `buildings_site_mapping`. Your output should contain the building name and the `json_agg` in that order. Order your final result by building name.

**Hint:** You should use a CTE to find the distinct building names of `real_estate_metadata` before applying necessary table joins. The DISTINCT keyword does not achieve the desired effect.

In [14]:
# %%sql
# select count(*)
# from buildings_site_mapping ta
# left join real_estate_metadata r on ta.building = r.building_name
# where r.building_name is null

In [15]:
# %%sql
# select building_name
# from real_estate_metadata r
# left join buildings_site_mapping ta on ta.building = r.building_name
# where ta.building is null

for each distinct real_estate_metadata.building_name value find the ones with multiple matches inbuildings_Site 

In [16]:
%%sql result_1c <<
with cte as 
(select DISTINCT building_name
    from real_estate_metadata)

select cte.building_name, json_agg(b) from buildings_site_mapping as b
INNER JOIN cte on cte.building_name = b.building
GROUP BY cte.building_name
HAVING count(cte.building_name) > 1
order by cte.building_name

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
4 rows affected.
Returning data to local variable result_1c


In [17]:
# Do not delete/edit this cell
result_1c.DataFrame().to_csv('results/result_1c.csv', index=False)

In [18]:
grader.check("q1c")

q1c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 1d

Do you see any issues with the schema given? Here are two worthwhile things to consider:
- Can you uniquely determine the building given the sensor data? Why?
- Could `buildings_site_mapping.building` be a valid foreign key pointing to `real_estate_metadata.building_name`?

<!--
BEGIN QUESTION
name: q1d
manual: true
points: 2
-->

There is an issue being that you may not determine a uniquely associated row in the metadata table given a row from the data table because since there is 5 places where the real_estate_metadata has the 2 or more of the same building_name so these cannot be uniquely identified. Even if we tried to set buildings_site_mapping.building to be a foreign key for real_estate_metadata.building_name, this wouldn't be valid because building_name is not a unique identifier for the real_estate_metadata table as determined by 1b. 


In [19]:
# %%sql
# SELECT COUNT(*), COUNT(DISTINCT(building_name))
# FROM real_estate_metadata
'''
Note: a foreign key column in a table points to a column with unique values in another table as a way of cross referencing the tables
'''

'\nNote: a foreign key column in a table points to a column with unique values in another table as a way of cross referencing the tables\n'

<!-- END QUESTION -->



## Question 2: Looking for Outliers in the Readings
Physical sensors like the ones generating this data are notorious for producing crazy outliers on occasion. In this section we'll do a little data cleaning of the outliers.

All the readings from all different kinds of sensors are mixed together in the `data` table. This hodgepodge of mixed readings is going to require us to do some extra work to look for outliers. Let's get started.

### Question 2a: Outlier Detection

Let's find the outlying values *for each sensor id*. We'll call something an outlier if it is **3 Hampel X84 intervals** away from the median. If 

Specifically, create a view `labeled_data` that contains all of the columns in `data` and adds three additional columns at the far right:
  - `median` containing the median using `percentile_disc`
  - `mad` containing the Median Absolute Deviation (MAD),
  - `outlier` that contains `true` for the outlier readings and `false` for the rest. **Also,** for data points where the mad is 0, set this to `false`.

Recall: Hampel x84: define outliers as  k∗1.4826  MADs from the median
mad_cte as 
(SELECT absdevs AS
(SELECT abs(data.id - median) as d
   FROM data, median_cte)
SELECT percentile_disc(0.5) WITHIN GROUP (ORDER BY d) as mad
 FROM 
  FROM absdevs)
  
group by the same sensor id's and THEN  calculate the mean for each

In [20]:
%%sql result_2a <<
CREATE OR REPLACE VIEW labeled_data AS
(with cte_median as 
(SELECT id, percentile_disc(0.5) WITHIN GROUP (ORDER BY data.value) as median
    FROM data
    GROUP BY data.id),

cte_mad as
(SELECT id, percentile_disc(0.5) WITHIN GROUP (ORDER BY d) as mad
    FROM (SELECT data.id, abs(data.value - median) as d
    FROM data
    INNER JOIN cte_median on cte_median.id = data.id) as deviations
    GROUP BY id)

select data.*, cte_median.median, cte_mad.mad, CASE WHEN value NOT BETWEEN (median - 3*1.4826*mad) AND (median + 3*1.4826*mad) AND mad != 0 THEN TRUE
 ELSE FALSE END as outlier
FROM data
INNER JOIN cte_median on cte_median.id = data.id
INNER JOIN cte_mad on cte_mad.id = data.id);

SELECT * FROM labeled_data WHERE outlier ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_2a


In [21]:
# Do not delete/edit this cell
result_2a.DataFrame().to_csv('results/result_2a.csv', index=False)

In [22]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b: Outlier Handling (Winsorization)

In this step we'll define a view `cleaned_data` over all the columns of `labeled_data` and one additional column on the far right called `clean_value`. This column will contain a copy of `data.value` if that value is not an outlier. For outliers, it should contain the value Winsorized to the nearest outlier boundary value (3 Hampel X84 intervals from the median). If the MAD is 0, then the cleaned value should be the same as the original value.

Reminder: Winsorizing
- replacing values with the k-percentile value
- k% tails contain the same repeated value
preserving the probability desnity of the tails, preferred to trimming if something downstream forbids null
"anything greater than the 99% percentile you will replace with the 99th percentile value
for a 90% winsorization get the 95% and 5% percentile and replace values outside of these bounds with the bound values (10/2 for each tail)

In [23]:
# %%sql
# SELECT *, 
#     CASE WHEN NOT outlier or mad = 0 THEN value
#         WHEN abs(value - (median - 3*1.4826*mad)) > abs(value - (median + 3*1.4826*mad))
#         THEN median + 3*1.4826*mad 
#     ELSE median - 3*1.4826*mad END as clean_value
# FROM labeled_data
# WHERE outlier
# LIMIT 100

In [24]:
%%sql result_2b <<
CREATE OR REPLACE VIEW cleaned_data AS
SELECT *, 
    CASE WHEN NOT outlier or mad = 0 THEN value
        WHEN abs(value - (median - 3*1.4826*mad)) > abs(value - (median + 3*1.4826*mad))
        THEN median + 3*1.4826*mad 
    ELSE median - 3*1.4826*mad END as clean_value
FROM labeled_data
;
SELECT * FROM cleaned_data WHERE outlier ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_2b


In [25]:
# Do not delete/edit this cell
result_2b.DataFrame().to_csv('results/result_2b.csv', index=False)

In [26]:
result_2b.DataFrame()

,time,id,value,median,mad,outlier,clean_value
0,2018-06-09 08:45:00+00:00,a47e9bd4-ab61-56d5-9e0e-2dce29a46dd3,98.700,37.2,12.500,True,92.797500
1,2018-06-07 00:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,812.600,727.5,16.200,True,799.554360
2,2018-06-07 18:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,807.300,727.5,16.200,True,799.554360
3,2018-06-07 18:15:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,800.000,727.5,16.200,True,799.554360
4,2018-06-07 18:30:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,819.500,727.5,16.200,True,799.554360
...,...,...,...,...,...,...,...
95,2018-06-10 10:30:03+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,0.468,25.0,3.571,True,9.116906
96,2018-06-10 20:15:01+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,42.857,25.0,3.571,True,40.883094
97,2018-06-11 11:00:17+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,0.420,25.0,3.571,True,9.116906
98,2018-06-11 15:00:07+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,42.857,25.0,3.571,True,40.883094


In [27]:
grader.check("q2b")

q2b results: All test cases passed!

<!-- BEGIN QUESTION -->

## Question 3: Entity Resolution

### Question 3a
There is a lot of mess in this dataset related to entity names. As a start, have a look at all of the distinct values in the `units` field of the `metadata` table. What do you notice about these values? Are there any duplicates?

<!--
BEGIN QUESTION
name: q3a
manual: true
points: 0
-->

It looks like sql is detecting the differences in capitalization as distinct values, for example it looks like there are many different versions of KWH as kWh or kWH which would be an issue for us if these are meant to represent the same value.

In [28]:
# %%sql
# SELECT DISTINCT units
# FROM metadata
# LIMIT 50

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 3b

Sometimes, entity resolution is as simple as a text transformation. For example, how many unique `units` values are there, and how many would there be if we ignored case (upper vs. lower case)? Your output should be a table with one row and two columns; the first column should contain the number of unique `units` values, and the second column should contain the number of unique `units` values if we ignored case. The two columns can have arbitrary names.

<!--
BEGIN QUESTION
name: q3bm
manual: true
points: 0
-->

In [29]:
%%sql result_3b <<
SELECT COUNT(DISTINCT units), COUNT(DISTINCT (upper(units)))
FROM metadata

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
1 rows affected.
Returning data to local variable result_3b


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [30]:
# Do not delete/edit this cell
result_3b.DataFrame().to_csv('results/result_3b.csv', index=False)

In [31]:
grader.check("q3b")

q3b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3c

Arguably we shouldn't care about these alternative unit labels, *as long as each sensor class uses a single value of `units` for all its sensor ids*. After all, maybe the capitalization means something to somebody!

Write a SQL query that returns single row with one column of value `true` if the condition in italics above holds, or a single row with one column of value `false` otherwise. Please do not hard code this query - we reserve the right to penalize your score if you do so.

<!--
BEGIN QUESTION
name: q3cm
manual: true
points: 0
-->

each sensor class uses a single value of units for all its sensor ids

In [32]:
# %%sql
# SELECT class, COUNT(DISTINCT units)
# FROM metadata
# GROUP BY class
# LIMIT 100

In [33]:
%%sql result_3c <<
with cte as (
SELECT class, COUNT(DISTINCT units)
FROM metadata
GROUP BY class)

SELECT CASE WHEN MAX(count) > 1 THEN FALSE ELSE TRUE END as case
FROM cte

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
1 rows affected.
Returning data to local variable result_3c


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [34]:
# Do not delete/edit this cell
result_3c.DataFrame().to_csv('results/result_3c.csv', index=False)

In [35]:
result_3c

case
True


In [36]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d

Moving on, have a look at the `real_estate_metadata` table---starting with the distinct values in the `location` field! What do you notice about these values?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 0
-->

In [37]:
%%sql
SELECT DISTINCT(location)
FROM real_estate_metadata

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
15 rows affected.


location
LOS ANGELES
FRANCISC O
SYSTEMWI DE
SAN DSAIENG O
FRANCISC SOAN
SANTA CRUZ
AG FIELD STAT
IRVINE
RIVERSIDE
BERKELEY


There appears to be spelling errors and inconsistencies in the distinct location types as well as repeated locations with different spellings. 

<!-- END QUESTION -->



### Question 3e

It turns out this table was the result of an [OCR scan](https://en.wikipedia.org/wiki/Optical_character_recognition). We'll just clean up the `location` column for now, and leave you to imagine the effort to do a full cleanup of all columns.

To provide some useful utility functions, we have preloaded Postgres' extension packages for "fuzzy" string matching and trigrams for you. You can use any of the string functions in those packages if you like ([as documented here for fuzzystrmatch](https://www.postgresql.org/docs/current/fuzzystrmatch.html) or [here for pg_trgm](https://www.postgresql.org/docs/current/pgtrgm.html)).

We also created a lookup table of canonical names, `uc_locations`.

Now, using any of the string functions you like (or none at all!), write a SQL query that returns the columns `(building_name, address, location, clean_location)` where `clean_location` contains the best match from `uc_locations.loc_name`. You may find that you can't clean up everything with the string functions, so your view may have to include some specific logic for cases in the data that have to be handled "manually". You can choose to do this question in whatever manner you wish as long as your query does not use `CREATE TABLE`, `INSERT INTO`, or `UPDATE`.

looking at the location from the metadata table and cleaning that up to match what is said in the uc_locations table

In [38]:
# %%sql
# SELECT *
# FROM uc_locations

In [39]:
# %%sql
# DROP VIEW IF EXISTS best_name

In [40]:
# %%sql
# CREATE OR REPLACE VIEW best_name AS
# SELECT loc_name
#     FROM real_estate_metadata r, uc_locations uc
#     WHERE location = 'SAN DSAIENG O' and levenshtein(uc.loc_name, location) <= 4;

# SELECT building_name, address, location, 
# CASE WHEN EXISTS (select *
#     from uc_locations uc
#     where uc.loc_name = r.location) THEN location
#     WHEN location ~ 'FRANCISC\s.*' THEN 'SAN FRANCISCO'
#     WHEN location ~ 'SYSTEMWI' THEN 'SYSTEMWIDE'
#     WHEN location = 'SAN DSAIENG O' THEN (select loc_name from best_name) END as clean_location
# FROM real_estate_metadata r,
# WHERE location = clean_location
# LIMIT 11;

In [41]:
%%sql result_3e <<
CREATE OR REPLACE VIEW best_name AS
SELECT loc_name
    FROM real_estate_metadata r, uc_locations uc
    WHERE location = 'SAN DSAIENG O' and levenshtein(uc.loc_name, location) <= 4;

SELECT building_name, address, location, 
CASE WHEN EXISTS (select *
    from uc_locations uc
    where uc.loc_name = r.location) THEN location
    WHEN location ~ 'FRANCISC\s.*' THEN 'SAN FRANCISCO'
    WHEN location ~ 'SYSTEMWI' THEN 'SYSTEMWIDE'
    WHEN location = 'SAN DSAIENG O' THEN (select loc_name from best_name) END as clean_location
FROM real_estate_metadata r;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
5276 rows affected.
Returning data to local variable result_3e


In [42]:
# Do not delete/edit this cell
result_3e.DataFrame().sort_values(['clean_location', 'building_name', 'address']).iloc[::10].to_csv('results/result_3e.csv', index=False)

In [43]:
grader.check("q3e")

q3e results: All test cases passed!

## Question 4: Interpolating Missing Data
Real-world data, real-world problems. Our sensors should be reporting every 15 minutes, but you can be sure that we're missing some data. Here we will fix it. It's a bit more involved than what we looked at in class!

### Question 4a: Finding missing readings
In the `data` table, the `id` column identifies a unique sensor. Sensor readings should be recorded every 15 minutes from every sensor. Are we missing any readings, and if so which ones? We will focus on readings that are separated by at least 30 minutes or more; readings that are \[0-30) minutes apart are considered to be fine.

To answer this question you'll need to read up a bit on [SQL timestamps](https://www.postgresql.org/docs/current/datatype-datetime.html) and [Functions for manipulating datetime types](https://www.postgresql.org/docs/current/functions-datetime.html). Have a particular look at the following:
- The [date_trunc](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC) function will quantize times to the nearest unit of your choosing. E.g. to round the `time` field to the nearest minute you can say `date_trunc('minute', time)`. **You'll need to quantize to minutes right away before you worry about missing readings.**
- There are various ways to enter constant intervals of time as strings. E.g. a 30 minute interval can be written as `interval '30 minutes'` or `'30 minutes'::interval`. See [date/time input](https://www.postgresql.org/docs/current/datatype-datetime.html#DATATYPE-INTERVAL-INPUT) for more info.
- You can do arithmetic on date/time types [as documented here](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC). That will handle all the weird periodicities of clocks and calendars for you. Pay attention to the input and output types of these functions!
- Alternatively, the [EXTRACT](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) function is sometimes handy. Note the special `EXTRACT(EPOCH FROM ...)` case. This converts a timestamp into an integer representing the number of seconds since midnight, 1/1/1970 (the dawn of [UNIX time](https://en.wikipedia.org/wiki/Unix_time)!)  You can do normal integer comparisons and arithmetic on the results.
- You will need to use the [lag](https://www.geeksforgeeks.org/postgresql-lag-function/) function as the window function.


Create a view called `gaps` that returns pairs of `data` tuples per sensor id that are separated by 30 minutes or more. The output should augment the `data` schema with three columns:
- `lagtime` is the quantized time of the previous reading for that sensor (relative to the current row for a particular row)
- `lagvalue` is the value of the previous reading for that sensor
- `timediff` is the difference in quantized time between this reading and the previous reading

plan: 
quantize to minutes for all dates then 
lagtime: date_trunc of previous reading using lag function lag over time and value
lagvalue: value of previous
timediff: can just subtract lagtime from current time lag extract lagtime and time
if case when

In [44]:
# %%sql
# with quantized as 
# (SELECT *, date_trunc('minute', time)
# FROM data),

# lagged as
# (SELECT date_trunc as time, id, value, LAG(date_trunc)  OVER (PARTITION BY id ORDER BY date_trunc) AS lagtime, LAG(value)  OVER (PARTITION BY id ORDER BY date_trunc) AS lagvalue, EXTRACT(epoch from date_trunc -  (LAG(date_trunc) OVER (PARTITION BY id ORDER BY date_trunc))) / 60 AS timediff
# FROM quantized)

# SELECT *
# FROM lagged
# WHERE timediff >= 30
# ORDER BY id, time
# LIMIT 10;

In [45]:
# %%sql
# DROP VIEW IF EXISTS gaps

In [46]:
%%sql result_4a <<
CREATE OR REPLACE VIEW gaps AS
with quantized as 
(SELECT *, date_trunc('minute', time)
FROM data),

lagged as
(SELECT date_trunc as time, id, value, LAG(date_trunc)  OVER (PARTITION BY id ORDER BY date_trunc) AS lagtime, 
 LAG(value)  OVER (PARTITION BY id ORDER BY date_trunc) AS lagvalue, 
 EXTRACT(epoch from date_trunc -  (LAG(date_trunc) OVER (PARTITION BY id ORDER BY date_trunc))) / 60 AS timediff
FROM quantized)

SELECT *
FROM lagged
WHERE timediff >= 30;


SELECT * FROM gaps ORDER BY id, time LIMIT 10;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
10 rows affected.
Returning data to local variable result_4a


In [47]:
# pd.read_csv('results/result_4a.csv')

In [48]:
# Do not delete/edit this cell
result_4a.DataFrame().to_csv('results/result_4a.csv', index=False)

In [49]:
result_4a.DataFrame()

,time,id,value,lagtime,lagvalue,timediff
0,2018-06-13 09:15:00+00:00,a3d47b1a-985e-5395-a6ee-719dad9b580f,0.056,2018-06-13 08:45:00+00:00,0.056,30.0000000000000000
1,2018-06-09 04:15:00+00:00,a3e8e405-0eed-59b6-8747-fe892a6f93de,280.800,2018-06-09 03:45:00+00:00,281.600,30.0000000000000000
2,2018-06-13 09:15:00+00:00,a46fb790-028c-5d17-a3e6-7d08daec0c03,281.460,2018-06-13 08:45:00+00:00,281.390,30.0000000000000000
3,2018-06-09 04:15:00+00:00,a470ac6d-d448-522b-9b74-8ed27f24de25,0.004,2018-06-09 03:45:00+00:00,0.006,30.0000000000000000
4,2018-06-07 00:45:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,27.273,2018-06-07 00:15:00+00:00,30.000,30.0000000000000000
5,2018-06-07 01:30:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,27.273,2018-06-07 01:00:00+00:00,23.077,30.0000000000000000
6,2018-06-07 17:45:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,21.429,2018-06-07 17:15:00+00:00,25.000,30.0000000000000000
7,2018-06-07 18:45:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,25.000,2018-06-07 18:15:00+00:00,30.000,30.0000000000000000
8,2018-06-07 19:15:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,20.000,2018-06-07 18:45:00+00:00,25.000,30.0000000000000000
9,2018-06-07 19:45:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,23.077,2018-06-07 19:15:00+00:00,20.000,30.0000000000000000


In [50]:
grader.check("q4a")

q4a results: All test cases passed!

### Question 4b: Creating tuples for the missing readings
Now we need to manufacture new tuples to fill in the gaps. For example, if you had a tuple from id `abc` timestamped at 1PM today and the next tuple in time from `abc` was timestamped at 1:45PM, you'll need to manufacture two new tuples with id `abc` and `NULL` values: one timestamped at 1:15PM and another timestamped at 1:30PM. We will worry about replacing the `NULL` values in the next step.

To manufacture tuples not related to stored data in the database, we'll need to use a *table-valued function* as we did in lecture 12 (when we manufactured data from a normal distribution). The table-valued function we want here is `generate_series` [(documented here)](https://www.postgresql.org/docs/current/functions-srf.html), which we will use to generate *and sequentially timestamp* the right number of tuples to match the number of tuples we found missing.

To get a feel for `generate_series`, consider the following simple query that generates a table of integers with intervals of size 3 between them.

In [51]:
%%sql
SELECT *
  FROM generate_series(1, 10, 3);

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
4 rows affected.


generate_series
1
4
7
10


Now, we can use `generate_series` in a `LATERAL JOIN` query: for each tuple on the left of the `LATERAL` it will produce a series based on the values of that tuple. So for example, we can generate 2 tuples for each tuple of `uc_locations` as follows:

In [52]:
%%sql
SELECT loc_id, loc_name, length(loc_name), newval
  FROM uc_locations u, 
       LATERAL generate_series(length(loc_name), length(loc_name) + 2, 2) AS newval;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
24 rows affected.


loc_id,loc_name,length,newval
1,AG FIELD STAT,13,13
1,AG FIELD STAT,13,15
2,BERKELEY,8,8
2,BERKELEY,8,10
3,DAVIS,5,5
3,DAVIS,5,7
4,IRVINE,6,6
4,IRVINE,6,8
5,LOS ANGELES,11,11
5,LOS ANGELES,11,13


Notice how the 2 values it generates are the length of the `loc_name`, and the length + 2. You might want to play with the query above to make sure you understand the documentation for `generate_series` and `LATERAL`.

Ok, on to your task!

Create a view `complete` that contains the tuples from `data` as well as new tuples that fill in any gaps greater than 30 minutes. Each gap should be filled by adding tuples in increments of 15 minutes from the *start* of the gap, with `NULL` as the value. You probably want to use your `gaps` view as well as `generate_series` to do this!

contains the tuples from the data as well as new tuples that fill in any gapes 30 minutes each gap should be filled by adding tuples in increments of 15 from the start of the gap

idea: using timediff from gap divided by 15 as x create however many series between time - 15*X to time with interval 15
lag time is the time from before so using that and having it start at lag_time, time, 15 is easier i think

In [53]:
%%sql 
DROP VIEW IF EXISTS complete

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.


[]

In [54]:
%%sql result_4b <<
CREATE OR REPLACE VIEW complete AS
with cte as
(SELECT *
  FROM gaps g, 
       LATERAL generate_series(lagtime + interval '15 minutes', time - interval '15 minutes', interval '15 minutes') AS newval)

SELECT * FROM data
UNION 
SELECT newval as time, id, NULL as value FROM cte
;

SELECT * FROM complete ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_4b


In [55]:
# Do not delete/edit this cell
result_4b.DataFrame().to_csv('results/result_4b.csv', index=False)

In [56]:
grader.check("q4b")

q4b results: All test cases passed!

### Question 4c: Linear Interpolation
*Note: If you struggled with the previous subparts of this problem, you can use our table `complete_provided` instead of your `complete` table in this subpart.*

Now, given the `complete` view or the `complete_provided` table, the remaining task is to do linear interpolation to fill in the missing values we manufactured in Step 2. We have code from Lecture 13 we can reuse here! In particular, your database already includes the UDA `coalesce_agg` we used in lecture (you can use it directly, there's no need to redefine it).

But note that in Lecture 13's example of linear interpolation we had a field called `feature_id` that had two convenient properties:
1. `feature_id` was used to order *all* the records in the table. By contrast, here the ordering we care about is the series of timestamps for each sensor `id` *independently*.
2. `feature_id` was a gap-free sequence of incrementing integers. Coupled with the previous point, that allowed us to use arithmetic on `feature_id` to calculate the distance between records in order in the "backward" pass. We don't have any such field handy here, so you'll have to find some other way (hint: window function!) to achieve the same effect where you need it.

These two points will require you to adapt the linear interpolation code from class to work here.

Create a view `likely_data` that contains all the tuples from `complete`, with an additional column called `interpolated` that contains a copy of `value` if it is non-NULL, otherwise an interpolated value based on linear interpolation *per sensor id over time*. The three cells below correspond to the forward, backward, and final passes from lecture.

1. may be relevant to use a partition by id for the timestamps to find the linear interpolation
a copy of value forward direction 


In [57]:
# %%sql
# with cte as (
# SELECT *, 
#     CASE WHEN value is not NULL THEN value 
#     ELSE coalesce(value, LEAD(value) OVER (PARTITION BY id ORDER BY time)) END as interpolated
# FROM complete_provided)

# SELECT *
# FROM cte
# WHERE  value is NULL
# LIMIT 5

- compute run, propagate run_start, get nextval into last row of run
- storing run, run_start, and intermediate value next val
- for every single row want an indication of what the previous non null value and what's the next non null
- going top down so we only know the previous non null value (coalesce does the previous)
- sum order by is a running sum, count all of the not null values that you see

In [58]:
%%sql
CREATE OR REPLACE VIEW forward AS
SELECT *, 
    SUM(CASE WHEN value IS NULL THEN 0 ELSE 1 END)
        OVER (PARTITION BY id ORDER BY time) AS run,
        coalesce_agg(value) OVER (PARTITION BY id ORDER BY time) AS run_start,
        CASE WHEN value IS NULL
        -- look ahead to the next row which hopefully has a not null value
            THEN lead(value, 1) OVER (PARTITION BY id ORDER BY time)
        ELSE NULL END AS nextval
    FROM complete_provided;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.


[]

Order by the entry rank but descending (going backwards through the table to get the first value using coalesce over reverse (gives us next val for every run). Works because when going forward with multiple rows of none values the last none would use lead to look ahead to a none NULL value. then you can backpropogate the nextval to everything within the same run.

In [59]:
# %%sql
# SELECT *, 
#     CASE WHEN value IS NOT NULL THEN value
#         ELSE coalesce_agg(nextval) OVER (PARTITION BY id, run ORDER BY time DESC)
#     END AS run_end,
#     count(*) OVER (PARTITION BY id, run) AS run_size,
#     count(*) OVER (PARTITION BY id, run ORDER BY time) AS run_rank
# FROM forward

In [60]:
%%sql
CREATE OR REPLACE VIEW backward AS
SELECT *, 
    CASE WHEN value IS NOT NULL THEN value
        ELSE coalesce_agg(nextval) OVER (PARTITION BY id, run ORDER BY time DESC)
    END AS run_end,
    count(*) OVER (PARTITION BY id, run) AS run_size,
    count(*) OVER (PARTITION BY id, run ORDER BY time) AS run_rank
FROM forward;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.


[]

In [61]:
# %%sql
# SELECT *, run_start + (run_rank-1) *((run_end - run_start)/(run_size))
#     AS interpolated
#     FROM backward
#     WHERE run_size > 2
#     LIMIT 10;

In [62]:
%%sql result_4c <<
CREATE OR REPLACE VIEW likely_data AS
SELECT *, run_start + (run_rank-1) *((run_end - run_start)/(run_size))
    AS interpolated
    FROM backward
;
SELECT * FROM likely_data WHERE run_size > 2 ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_4c


In [63]:
# Do not delete/edit this cell
result_4c.DataFrame().to_csv('results/result_4c.csv', index=False)

In [64]:
grader.check("q4c")

q4c results: All test cases passed!

## Congratulations! You have finished Project 3.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**For submission on Gradescope, you will need to submit BOTH the proj3.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj3.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [65]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

updating: results/ (stored 0%)
updating: results/result_1b.csv (deflated 78%)
updating: results/result_1c.csv (deflated 57%)
updating: results/result_2a.csv (deflated 92%)
updating: results/result_2b.csv (deflated 92%)
updating: results/result_3b.csv (deflated 6%)
updating: results/result_3c.csv (stored 0%)
updating: results/result_3e.csv (deflated 76%)
updating: results/result_4a.csv (deflated 75%)
updating: results/result_4b.csv (deflated 91%)
updating: results/result_4c.csv (deflated 84%)


/home/jovyan/fa22/proj/proj3/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [66]:
grader.check_all()

q1b results: All test cases passed!

q1c results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q3e results: All test cases passed!

q4a results: All test cases passed!

q4b results: All test cases passed!

q4c results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [67]:
# Save your notebook first, then run this cell to export your submission.
grader.export()